In [1]:
%reset -f

In [2]:
import ee
ee.Authenticate()


In [7]:
ee.Initialize(project='ee-andrewfullhart')

prism_ic = ee.ImageCollection('OREGONSTATE/PRISM/AN81m')
first_im = prism_ic.first().select('ppt')
scale = first_im.projection().nominalScale().getInfo()

cover_ic = ee.ImageCollection('projects/rap-data-365417/assets/vegetation-cover-v3')
prod_ic = ee.ImageCollection('projects/rap-data-365417/assets/npp-partitioned-v3')

years_list= ee.List([1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005])
label_order = ee.List(['RAP_afg', 'RAP_pfg', 'RAP_bgr', 'RAP_ltr', 'RAP_shr', 'RAP_tre', 'RAP_afgNPP', 'RAP_pfgNPP', 'RAP_shrNPP', 'RAP_treNPP'])
area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

im = cover_ic.first()
proj = im.projection().getInfo()

transform = [
  proj['transform'][0],
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  proj['transform'][4],
  proj['transform'][5],
]

transform_new = [
  0.0083333333,
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  0.0083333333,
  proj['transform'][5],
]

proj = im.projection()

def clip_fnA(yrobj):
  year = ee.Number(yrobj)
  start = ee.Date.fromYMD(year, 1, 1)
  end = ee.Date.fromYMD(year.add(1), 1, 1)
  yr_im = cover_ic.filterDate(start, end).select(['AFG', 'PFG', 'BGR', 'LTR', 'SHR', 'TRE']).first()
  im = yr_im.setDefaultProjection('EPSG:4326', transform)
  im = im.reproject(crs=proj.crs(), crsTransform=transform_new)
  clip_im = ee.Image(im).clip(area_shp)
  return clip_im

c_ic = ee.ImageCollection(years_list.map(clip_fnA))

def clip_fnB(yrobj):
  year = ee.Number(yrobj)
  start = ee.Date.fromYMD(year, 1, 1)
  end = ee.Date.fromYMD(year.add(1), 1, 1)
  yr_im = prod_ic.filterDate(start, end).select(['afgNPP', 'pfgNPP', 'shrNPP', 'treNPP']).first()
  im = yr_im.setDefaultProjection('EPSG:4326', transform)
  im = im.reproject(crs=proj.crs(), crsTransform=transform_new)
  clip_im = ee.Image(im).clip(area_shp)
  return clip_im

p_ic = ee.ImageCollection(years_list.map(clip_fnB))



cavg_im = c_ic.reduce(ee.Reducer.mean())
afg_im = cavg_im.select('AFG_mean')
pfg_im = cavg_im.select('PFG_mean')
bgr_im = cavg_im.select('BGR_mean')
ltr_im = cavg_im.select('LTR_mean')
shr_im = cavg_im.select('SHR_mean')
tre_im = cavg_im.select('TRE_mean')

pavg_im = p_ic.reduce(ee.Reducer.mean())
afgnpp_im = pavg_im.select('afgNPP_mean')
pfgnpp_im = pavg_im.select('pfgNPP_mean')
shrnpp_im = pavg_im.select('shrNPP_mean')
trenpp_im = pavg_im.select('treNPP_mean')


map_ic = ee.ImageCollection([afg_im, pfg_im, bgr_im, ltr_im, shr_im, tre_im, afgnpp_im, pfgnpp_im, shrnpp_im, trenpp_im])
map_ic_list = map_ic.toList(map_ic.size())

for i in range(map_ic.size().getInfo()):

  eimage = ee.Image(map_ic_list.get(i))
  label = ee.String(label_order.get(i)).getInfo()

  task = ee.batch.Export.image.toDrive(
    image=eimage,
    description=label,
    folder='GEE_Downloads',
    region=area_shp,
    scale=scale,
    crs='EPSG:4326',
    maxPixels=1e13
  )

  task.start()

